# Forest Growth Simulation with pyFVS

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mihiarc/fiatools/blob/main/tutorials/03_forest_growth_simulation_with_pyfvs.ipynb)
[![FIAtools](https://img.shields.io/badge/FIAtools-Ecosystem-2E7D32)](https://fiatools.org)

This tutorial shows you how to simulate forest growth using **pyFVS**, a Python implementation of the USDA Forest Service's Forest Vegetation Simulator.

## What You'll Learn

- What FVS is and how growth models work
- How to create and grow virtual forest stands
- How to simulate timber management scenarios
- How to generate yield tables and projections
- How to compare management strategies

## Prerequisites

- Python 3.9 or higher
- Basic understanding of forestry concepts (DBH, site index, basal area)

---

## 1. What is FVS?

The [Forest Vegetation Simulator (FVS)](https://www.fs.usda.gov/fvs/) is the USDA Forest Service's primary tool for projecting forest stand dynamics. It:

- **Simulates individual tree growth** using empirical growth equations
- **Models mortality** from competition and natural causes
- **Projects stand development** over time (typically 50-100 years)
- **Evaluates management alternatives** (thinning, harvesting, planting)

### Why pyFVS?

The official FVS is a Fortran program with a Windows GUI. **pyFVS** provides:

- **Pure Python implementation** - runs anywhere
- **Programmatic API** - script thousands of scenarios
- **Modern architecture** - clean, maintainable code
- **Southern pine focus** - loblolly, longleaf, slash, shortleaf

## 2. Installation

Install pyFVS from PyPI:

In [ ]:
# Install pyFVS (uncomment to run)
# !pip install pyfvs-fia

## 3. Creating a Forest Stand

The `Stand` class represents a forest stand with trees you can grow over time:

In [ ]:
from pyfvs import Stand

# Create a planted loblolly pine stand
stand = Stand.initialize_planted(
    trees_per_acre=500,    # Planting density
    site_index=70,         # Site productivity (base age 25)
    species='LP'           # Loblolly pine
)

print(f"Stand created with {stand.get_tpa()} trees per acre")
print(f"Site index: {stand.site_index}")

### Supported Species

pyFVS currently supports Southern yellow pines:

| Code | Species | Common Use |
|------|---------|------------|
| LP | Loblolly pine | Most common planted pine |
| SP | Shortleaf pine | Natural regeneration |
| LL | Longleaf pine | Restoration, fire-adapted |
| SA | Slash pine | Pulpwood, turpentine |

## 4. Growing the Stand

Use the `grow()` method to project the stand forward in time:

In [ ]:
# Grow for 25 years
stand.grow(years=25)

# Get current metrics
metrics = stand.get_metrics()
print(f"\nAfter 25 years:")
print(f"  Trees per acre: {metrics['tpa']:.0f}")
print(f"  Average DBH: {metrics['qmd']:.1f} inches")
print(f"  Basal area: {metrics['basal_area']:.0f} sq ft/acre")
print(f"  Volume: {metrics['volume']:.0f} cubic ft/acre")

### Key Metrics

| Metric | Description | Typical Range |
|--------|-------------|---------------|
| TPA | Trees per acre | 50-800 |
| QMD | Quadratic mean diameter | 4-18 inches |
| Basal area | Cross-sectional area at 4.5 ft | 50-200 sq ft/acre |
| Volume | Merchantable stem volume | 500-8000 cuft/acre |

## 5. Understanding Site Index

Site index is a measure of site productivity - the expected height of dominant trees at a base age (usually 25 or 50 years):

In [ ]:
# Compare different site qualities
for si in [50, 60, 70, 80]:
    s = Stand.initialize_planted(500, si, 'LP')
    s.grow(years=25)
    m = s.get_metrics()
    print(f"SI={si}: Volume={m['volume']:.0f} cuft/acre, QMD={m['qmd']:.1f} in")

### Site Index Guidelines

| Site Index (base 25) | Site Quality | Typical Locations |
|---------------------|--------------|-------------------|
| 50-55 | Poor | Dry ridges, sandy soils |
| 60-65 | Average | Upland sites |
| 70-75 | Good | Well-drained bottomlands |
| 80+ | Excellent | Prime alluvial sites |

## 6. Simulating Thinning

Thinning removes trees to reduce competition and concentrate growth on remaining trees:

In [ ]:
# Create stand and grow to thinning age
stand = Stand.initialize_planted(500, 70, 'LP')
stand.grow(years=15)

print(f"Before thinning: {stand.get_tpa():.0f} TPA, {stand.get_metrics()['qmd']:.1f}\" QMD")

# Thin from below (remove smallest trees) to 200 TPA
stand.thin_from_below(target_tpa=200)

print(f"After thinning: {stand.get_tpa():.0f} TPA, {stand.get_metrics()['qmd']:.1f}\" QMD")

# Continue growing
stand.grow(years=10)
print(f"After 10 more years: {stand.get_tpa():.0f} TPA, {stand.get_metrics()['qmd']:.1f}\" QMD")

### Thinning Methods

| Method | Description | Use Case |
|--------|-------------|----------|
| `thin_from_below()` | Remove smallest trees | Most common, improves average size |
| `thin_from_above()` | Remove largest trees | Uncommon, harvests value early |
| `thin_proportional()` | Remove across all sizes | Maintains structure |

## 7. Generating Yield Tables

Create a traditional yield table showing stand development over time:

In [ ]:
from pyfvs import Stand

# Initialize stand
stand = Stand.initialize_planted(500, 70, 'LP')

# Generate yield table
yield_table = []
for age in range(0, 35, 5):
    if age > 0:
        stand.grow(years=5)
    m = stand.get_metrics()
    yield_table.append({
        'Age': age,
        'TPA': int(m['tpa']),
        'QMD': round(m['qmd'], 1),
        'BA': int(m['basal_area']),
        'Volume': int(m['volume'])
    })

# Display as table
import pandas as pd
df = pd.DataFrame(yield_table)
print("\nYield Table: Loblolly Pine, SI=70, 500 TPA initial")
print(df.to_string(index=False))

## 8. Comparing Management Scenarios

Compare different management strategies to optimize objectives:

In [ ]:
def simulate_scenario(name, initial_tpa, thin_age=None, thin_target=None):
    """Simulate a management scenario and return final metrics."""
    stand = Stand.initialize_planted(initial_tpa, 70, 'LP')
    
    if thin_age:
        stand.grow(years=thin_age)
        stand.thin_from_below(target_tpa=thin_target)
        stand.grow(years=30 - thin_age)
    else:
        stand.grow(years=30)
    
    m = stand.get_metrics()
    return {
        'Scenario': name,
        'Final TPA': int(m['tpa']),
        'Avg DBH': round(m['qmd'], 1),
        'Volume': int(m['volume'])
    }

# Compare scenarios
scenarios = [
    simulate_scenario("No thinning (500 TPA)", 500),
    simulate_scenario("Thin at 15 to 200", 500, thin_age=15, thin_target=200),
    simulate_scenario("Thin at 12 to 150", 500, thin_age=12, thin_target=150),
    simulate_scenario("Wide spacing (300 TPA)", 300),
]

print("\nManagement Scenario Comparison (30-year rotation)")
print(pd.DataFrame(scenarios).to_string(index=False))

## 9. Ecological Units

Growth varies by ecological region. Use the `ecounit` parameter to adjust:

In [ ]:
# Compare growth across regions
for ecounit in ['232', '231L', 'M231']:
    stand = Stand.initialize_planted(500, 70, 'LP', ecounit=ecounit)
    stand.grow(years=25)
    m = stand.get_metrics()
    print(f"Ecounit {ecounit}: Volume={m['volume']:.0f} cuft/acre")

### Ecological Unit Effects

| Ecounit | Region | Growth Modifier |
|---------|--------|----------------|
| 232 | Georgia Coastal Plain | Base (1.0x) |
| 231L | Gulf Coastal Lowland | ~1.3x |
| M231 | Southern Appalachian | ~2.2x |

## 10. Complete Example: Timber Investment Analysis

In [ ]:
def timber_investment_analysis(site_index: int, rotation_length: int):
    """Analyze timber investment returns for a loblolly pine plantation."""
    from pyfvs import Stand
    
    # Assumptions
    planting_cost = 200  # $/acre
    pulpwood_price = 10  # $/ton
    sawtimber_price = 25  # $/ton (trees > 10" DBH)
    tons_per_cuft = 0.02  # Conversion factor
    
    # Create and grow stand
    stand = Stand.initialize_planted(500, site_index, 'LP', ecounit='M231')
    stand.grow(years=rotation_length)
    
    # Get final metrics
    metrics = stand.get_metrics()
    total_volume = metrics['volume']
    avg_dbh = metrics['qmd']
    
    # Calculate revenue (simplified)
    total_tons = total_volume * tons_per_cuft
    if avg_dbh >= 10:
        revenue = total_tons * sawtimber_price
        product = "sawtimber"
    else:
        revenue = total_tons * pulpwood_price
        product = "pulpwood"
    
    net_revenue = revenue - planting_cost
    annual_return = net_revenue / rotation_length
    
    print(f"\n{'='*50}")
    print(f"Timber Investment Analysis")
    print(f"{'='*50}")
    print(f"Site Index: {site_index}")
    print(f"Rotation: {rotation_length} years")
    print(f"\nFinal Stand:")
    print(f"  Volume: {total_volume:,.0f} cuft/acre")
    print(f"  Avg DBH: {avg_dbh:.1f} inches")
    print(f"  Product: {product}")
    print(f"\nFinancials:")
    print(f"  Gross revenue: ${revenue:,.0f}/acre")
    print(f"  Net revenue: ${net_revenue:,.0f}/acre")
    print(f"  Annual return: ${annual_return:,.0f}/acre/year")
    
    return net_revenue

# Uncomment to run:
# timber_investment_analysis(site_index=70, rotation_length=30)

## 11. Next Steps

Now that you can simulate forest growth, explore more:

### Advanced pyFVS
- Multiple thinning entries
- Mixed species stands
- Carbon accounting
- Custom mortality models

### Integration with FIAtools
- Use **pyFIA** to get real stand data as model inputs
- Use **gridFIA** to identify site conditions for simulations
- Ask **askFIA** questions about forest management

### Resources
- [pyFVS Documentation](https://mihiarc.github.io/pyfvs/)
- [FVS Official Site](https://www.fs.usda.gov/fvs/)
- [Southern Variant Guide](https://www.fs.usda.gov/fvs/documents/)

---

**Questions or feedback?** Open an issue on [GitHub](https://github.com/mihiarc/pyfvs/issues).